# Moduel 15.02 Data Cleaning I: Variable Types

In [9]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'youtube'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

youtube_df = pd.read_sql_query('select * from youtube',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

youtube_df.info()
youtube_df.head() # shows more than 1 int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           5000 non-null   object
 1   Grade          5000 non-null   object
 2   Channel name   5000 non-null   object
 3   Video Uploads  5000 non-null   object
 4   Subscribers    5000 non-null   object
 5   Video views    5000 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views
0,1st,A++,Zee TV,82757,18752951,20869786591
1,2nd,A++,T-Series,12661,61196302,47548839843
2,3rd,A++,Cocomelon - Nursery Rhymes,373,19238251,9793305082
3,4th,A++,SET India,27323,31180559,22675948293
4,5th,A++,WWE,36756,32852346,26273668433


In [10]:
# shows there are blanks with '--'
youtube_df[(youtube_df["Video Uploads"].str.strip() == "--") | (youtube_df["Subscribers"].str.strip() == "--")]

,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views
17,18th,A+,Vlad and Nikita,53,--,1428274554
108,109th,A,BIGFUN,373,--,941376171
115,116th,A,Bee Kids Games - Children TV,740,--,414535723
142,143rd,A,ChiChi TV Siêu Nhân,421,--,2600394871
143,144th,A,MusicTalentNow,1487,--,3252752212
...,...,...,...,...,...,...
4941,"4,942nd",B+,GMTV,183,--,127080542
4948,"4,949th",B+,Keivon ToysReview,468,--,481568513
4956,"4,957th",B+,CLICKNEWS,2661,--,139940815
4961,"4,962nd",B+,ONE Championship,905,--,109836654


In [11]:
youtube_df.nunique() # unique values in each column

Rank             5000
Grade               6
Channel name     4993
Video Uploads    2286
Subscribers      4612
Video views      5000
dtype: int64

## Grouping Numbers to Categories

In [12]:
# This method returns group numbers 
# given video views
def categorize_video_views(views_num):
    if views_num >= 1000000000:
        return 1
    elif views_num >= 100000000:
        return 2
    else:
        return 3

# Use the pandas `apply()` method by calling the function above
youtube_df['views_group'] = youtube_df['Video views'].apply(categorize_video_views)

# Examine how many observations you have in each group
print(youtube_df.groupby("views_group")["Video views"].count())

views_group
1    1399
2    2846
3     755
Name: Video views, dtype: int64


# Moduel 15.03 Data Cleaning I: Missing Values

In [13]:
youtube_df.isnull().sum()

Rank             0
Grade            0
Channel name     0
Video Uploads    0
Subscribers      0
Video views      0
views_group      0
dtype: int64

In [6]:
# print issues with turning data into an integer

for column_name in ["Video Uploads", "Subscribers"]:
    print("These are the problematic values for the variable: {}".format(column_name))
    prob_vars = []
    for value in youtube_df[column_name]:
        try:
            float(value)
        except:
            prob_vars.append(value)
    print(np.unique(prob_vars))

These are the problematic values for the variable: Video Uploads
['--']
These are the problematic values for the variable: Subscribers
['-- ']


In [14]:
# Replace `--` values with empty strings

youtube_df["Video Uploads"] = youtube_df["Video Uploads"].apply(str.strip).replace("--", np.nan)
youtube_df["Video Uploads"] = pd.to_numeric(youtube_df["Video Uploads"], downcast="float")

youtube_df["Subscribers"] = youtube_df["Subscribers"].apply(str.strip).replace("--", np.nan)
youtube_df["Subscribers"] = pd.to_numeric(youtube_df["Subscribers"], downcast="float")

In [22]:
pd.DataFrame(youtube_df.isnull().sum().DataFrame())

AttributeError: 'Series' object has no attribute 'DataFrame'

In [16]:
# print issues with turning data into an integer

for column_name in ["Video Uploads", "Subscribers"]:
    print("These are the problematic values for the variable: {}".format(column_name))
    prob_vars = []
    for value in youtube_df[column_name]:
        try:
            float(value)
        except:
            prob_vars.append(value)
    print(np.unique(prob_vars))

These are the problematic values for the variable: Video Uploads
[]
These are the problematic values for the variable: Subscribers
[]
